In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import os
import json

In [2]:
base_url = 'https://forums.hardwarezone.com.sg/'
r = requests.get(base_url)

In [3]:
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a', {'class': 'subNodeLink'})
a.extend(soup.find_all('a', {'data-shortcut': 'node-description'}))
a = [a_.get('href') for a_ in a]
topics = sorted(list(set([a_ for a_ in a if '/link-forums' not in a_])))

In [4]:
!mkdir topics

mkdir: cannot create directory ‘topics’: File exists


In [12]:
topics

['/forums/android-user-group.323/',
 '/forums/apple-clinic.5/',
 '/forums/apple-watch.374/',
 '/forums/australia.317/',
 '/forums/campus-zone.180/',
 '/forums/cars-cars.204/',
 '/forums/certified-systems-it-security-and-network-trainin.68/',
 '/forums/chill-out-hangout-den.234/',
 '/forums/china.316/',
 '/forums/console-gaming.383/',
 '/forums/cpu-ram-mainboard-bazaar.199/',
 '/forums/credit-cards-line-of-credit-facilities.243/',
 '/forums/current-affairs-lounge.17/',
 '/forums/degree-programs-and-courses.70/',
 '/forums/design-visual-art-gallery-sig.123/',
 '/forums/digital-cameras-photography.12/',
 '/forums/diploma-programs-and-courses.67/',
 '/forums/eat-drink-man-woman.16/',
 '/forums/electronics-bazaar.259/',
 '/forums/employment-office.22/',
 '/forums/english-premier-league.179/',
 '/forums/europe.277/',
 '/forums/events-entertainment-celebrity-buzz.242/',
 '/forums/fashion-bazaar.308/',
 '/forums/fashion-grooming.235/',
 '/forums/football-and-sports-arena.20/',
 '/forums/gaming

In [5]:
max_worker = 20

def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except Exception as e:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    try:
        max_page = max([int(a_.split('page-')[-1]) for a_ in a if '/forums/' in a_ and '/page-' in a_])
    except:
        max_page = -1
    a = [a_ for a_ in a if a_.startswith('/threads')]
    filtered = []
    for a_ in a:
        splitted = a_.split('/')
        if len(splitted) == 4 and splitted[-1] == '':
            filtered.append(a_)
    return filtered, max_page

In [6]:
a_ = topics[0]
base_url = f'https://forums.hardwarezone.com.sg{a_}'
    
filtered, max_page = get_href(base_url)

In [7]:
base_url

'https://forums.hardwarezone.com.sg/forums/android-user-group.323/'

In [8]:
filtered, max_page

(['/threads/batteries-charging-and-you-why-no-100-charge-on-your-phone.3070181/',
  '/threads/batteries-charging-and-you-why-no-100-charge-on-your-phone.3070181/',
  '/threads/switching-from-iphone-to-android.4853305/',
  '/threads/switching-from-iphone-to-android.4853305/',
  '/threads/read-before-you-post-rules-in-android-user-group.3069765/',
  '/threads/read-before-you-post-rules-in-android-user-group.3069765/',
  '/threads/noobs-a-dummies-guide-to-android-terminology.3071051/',
  '/threads/noobs-a-dummies-guide-to-android-terminology.3071051/',
  '/threads/google-pixel-7a.6825753/',
  '/threads/google-pixel-7a.6825753/',
  '/threads/google-tensor-g3-and-beyond.6900061/',
  '/threads/google-tensor-g3-and-beyond.6900061/',
  '/threads/exynos-2400-leaps-over-the-adreno-750-of-snapdragon-8-gen-3.6896773/',
  '/threads/exynos-2400-leaps-over-the-adreno-750-of-snapdragon-8-gen-3.6896773/',
  '/threads/htc-u23-pro-incoming-upper-midranger.6901067/',
  '/threads/htc-u23-pro-incoming-upper

In [11]:
for i in tqdm(range(len(topics))):
    a_ = topics[i]
    filename = os.path.join('topics', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    base_url = f'https://forums.hardwarezone.com.sg{a_}'
    
    filtered, max_page = get_href(base_url)
    
    for i in tqdm(range(2, min(5000, max_page + 1), max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{base_url}page-{a}' for a in aranged]

        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    filtered = list(set(filtered))
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

 77%|███████████████████████████████▍         | 192/250 [12:07<06:01,  6.23s/it]


 20%|████████▊                                   | 2/10 [00:02<00:09,  1.15s/it]


 58%|████████████████████████▊                  | 56/97 [27:05<08:22, 12.26s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.07s/it]


 86%|████████████████████████████████████▊      | 83/97 [29:22<01:03,  4.50s/it]
0it [00:00, ?it/s]
 14%|██████▎                                     | 2/14 [00:02<00:14,  1.23s/it]


100%|███████████████████████████████████████████| 97/97 [30:49<00:00, 19.07s/it]


In [13]:
!du -hs topics

20M	topics


In [15]:
# !zip -r topics.zip topics